 # I. Création de la base de donées annotées

## I.1. Capture des photos

In [1]:
import cv2
from cvzone import HandTrackingModule

In [ ]:
#changer le label en fonction du dataset
label = input("Entree le label : ")
cnt_img = 100
detector = HandTrackingModule.HandDetector()

In [ ]:
#Initialisation de la webcam
capture = cv2.VideoCapture(0)

while True:
    #capture d'une image du flux de la webcam
    ret,img = capture.read()
    img_copy = img.copy()
    hands, img = detector.findHands(img)

    if not ret:
        print("Erreur lors de la lecture de img")
        break

    #Affichage de l'image
    cv2.imshow("Image", img)

    key = cv2.waitKey(1)
    # ESC 
    if key%256 == 27:
        print("ESC, fermeture...")
        break
    # ESPACE
    elif key%256 == 32:

        bbox_value = hands[0].get('bbox')

        #Ecrit le roi dans le fichier
        roi = img_copy[bbox_value[1]:bbox_value[1] + bbox_value[3], bbox_value[0]:bbox_value[0] + bbox_value[2]]

        img_name = "image_final/{0}/{0}_{1}.png".format(label,cnt_img)
        cv2.imwrite(img_name, roi)
        print("{} ecrit!".format(img_name))
        cnt_img += 1


capture.release()
cv2.destroyAllWindows()   

## I.2. Filtre gaussien

In [2]:
import cv2
import numpy as np
import os

In [3]:

def gaussian_filter(img):
    # Charger l'image
    image = cv2.imread(img)
    # Appliquer le filtre Gaussien
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    # Enregistrer l'image filtrée
    cv2.imwrite(img, blurred)

In [4]:

root_dir = 'image_final'
for sub_dir in os.listdir(root_dir):
    sub_dir_path = os.path.join(root_dir, sub_dir)
    for filename in os.listdir(sub_dir_path):
        img = os.path.join(sub_dir_path, filename)
        gaussian_filter(img)


## I.2. Data Augmentation

In [5]:
import Augmentor
import os

In [6]:
root_dir = 'image_final'

In [7]:
for sub_dir in os.listdir(root_dir):
    sub_dir_path = os.path.join(root_dir, sub_dir)

    p = Augmentor.Pipeline(sub_dir_path)

    p.zoom(probability=0.3,min_factor=0.8,max_factor=1.5)
    p.flip_top_bottom(probability=0.4)
    p.random_brightness(probability=0.3,min_factor=0.3,max_factor=1.2)
    p.random_distortion(probability=1,grid_width=4,grid_height=4,magnitude=8)

    p.sample(500)

    output_dir = os.path.join(sub_dir_path, 'output')
    if os.path.isdir(output_dir):
        for filename in os.listdir(output_dir):
            src = os.path.join(output_dir, filename)
            dst = os.path.join(sub_dir_path, filename)
            os.rename(src, dst)
        os.rmdir(output_dir)

Initialised with 91 image(s) found.
Output directory set to image_final\A\output.

Processing <PIL.Image.Image image mode=RGB size=127x90 at 0x1B05F621780>: 100%|██████████| 500/500 [00:02<00:00, 177.37 Samples/s] 


Initialised with 79 image(s) found.
Output directory set to image_final\B\output.

Processing <PIL.Image.Image image mode=RGB size=89x115 at 0x1B05F4918A0>: 100%|██████████| 500/500 [00:01<00:00, 265.12 Samples/s] 


Initialised with 84 image(s) found.
Output directory set to image_final\C\output.

...

Processing <PIL.Image.Image image mode=RGB size=56x90 at 0x1B05F60ABC0>: 100%|██████████| 500/500 [00:01<00:00, 314.75 Samples/s]  


Initialised with 71 image(s) found.
Output directory set to image_final\V\output.

Processing <PIL.Image.Image image mode=RGB size=97x215 at 0x1B05F4AE500>: 100%|██████████| 500/500 [00:03<00:00, 141.08 Samples/s] 


Initialised with 71 image(s) found.
Output directory set to image_final\W\output.

Processing <PIL.Image.Image image mode=RGB size=86x108 at 0x1B05F608E20>: 100%|██████████| 500/500 [00:01<00:00, 304.96 Samples/s] 


# II. Création et entrainement d'un réseau de neurones

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import cv2
import pydot

In [9]:
# Chargement du dataset X->images y-> labels
data_dir = "image_final/"
labels = sorted(os.listdir(data_dir))

X = []
y = []
for idx, label in enumerate(labels):
  for file in os.listdir(data_dir + '/'+label):
    filepath = data_dir +'/'+ label + "/" + file
    img = cv2.resize(cv2.imread(filepath),(50,50))
    X.append(img)
    y.append(idx)
X = np.asarray(X)
y = np.asarray(y)
print(X.shape, y.shape)

(5742, 50, 50, 3) (5742,)


In [10]:
#Decoupage du dataset 80% training, 20% test
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.8)

In [11]:
#Normalisation des pixels et transformation en vecteur binaire
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
X_train = X_train / 255.
X_test = X_test/ 255.

In [12]:
# Création du model
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu', input_shape=(50,50,3)),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 16)        2320      
                                                                 
 conv2d_2 (Conv2D)           (None, 44, 44, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 16)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 32)        4640      
                                                                 
 conv2d_4 (Conv2D)           (None, 18, 18, 32)        9248      
                                                        

In [13]:
#Compilation 
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer,
               loss = 'categorical_crossentropy',
               metrics=['accuracy'],)

In [14]:
#Entrainement
history = model.fit(X_train, Y_train, epochs=36, verbose=1,
                validation_data=(X_test, Y_test))

Epoch 1/36
36/36 [==============================] - 8s 135ms/step - loss: 2.3035 - accuracy: 0.0967 - val_loss: 2.3020 - val_accuracy: 0.1012
Epoch 2/36
36/36 [==============================] - 4s 125ms/step - loss: 2.3055 - accuracy: 0.1098 - val_loss: 2.2971 - val_accuracy: 0.1119
Epoch 3/36
36/36 [==============================] - 4s 125ms/step - loss: 2.2381 - accuracy: 0.1629 - val_loss: 2.0903 - val_accuracy: 0.1959
Epoch 4/36
36/36 [==============================] - 4s 126ms/step - loss: 2.0032 - accuracy: 0.2779 - val_loss: 1.8387 - val_accuracy: 0.3167
Epoch 5/36
36/36 [==============================] - 5s 129ms/step - loss: 1.7085 - accuracy: 0.4007 - val_loss: 1.6678 - val_accuracy: 0.4462
Epoch 6/36
36/36 [==============================] - 5s 130ms/step - loss: 1.4491 - accuracy: 0.4948 - val_loss: 1.2334 - val_accuracy: 0.5805
Epoch 7/36
36/36 [==============================] - 5s 130ms/step - loss: 1.0758 - accuracy: 0.6132 - val_loss: 1.0549 - val_accuracy: 0.6448
Epoch 

In [15]:
model.save("model_save/model_final.h5")

# III. Détéction

## III.1. Détéction avec des images

In [16]:
from keras.models import load_model
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


#Chargement du modèle
loaded_model = load_model("model_save/model_final.h5")


classData = {
    0 : "A",
    1 : "B",
    2 : "C",
    3 : "G",
    4 : "H",
    5 : "I",
    6 : "L",
    7 : 'R',
    8 : 'V',
    9 : 'W'
}
#Chargement de l'image
new_image = cv2.imread("img_val/Val_0.png")


#Prétraitements identique aux images d'entraînement
new_image = cv2.GaussianBlur(new_image, (5, 5), 0)   
new_image = cv2.resize(new_image, (50, 50))
new_image = np.array(new_image) / 255.0
new_image = np.expand_dims(new_image, axis=0)
# Prediction
predictions = loaded_model.predict(new_image)
# Récupère la classe avec la plus grande probabilité
class_index = np.argmax(predictions[0])
#Affichage
results = predictions[0].tolist()
print(classData[class_index])


1/1 [==============================] - 0s 118ms/step
B


## III.2. Détéction avec un flux vidéo

In [ ]:
from keras.models import load_model
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from cvzone import HandTrackingModule


#Initialisation de la webcamA
capture = cv2.VideoCapture(0)
detector = HandTrackingModule.HandDetector()
loaded_model = load_model("model_save/model_final.h5")

classData = {
    0 : "A",
    1 : "B",
    2 : "C",
    3 : "G",
    4 : "H",
    5 : "I",
    6 : "L",
    7 : 'R',
    8 : 'V',
    9 : 'W'
}
while True:
    #capture d'une image du flux de la webcam
    # ret,img = capture.read()
    ret,img = capture.read()
    img_copy = img.copy()
    hands, img = detector.findHands(img)
    
    if not ret:
        print("Erreur lors de la lecture de img")
        break

    key = cv2.waitKey(1)
    # ESC 
    if key%256 == 27:
        print("ESC, fermeture...")
        break
    # ESPACE
    # elif key%256 == 32:
    if hands != []:
        #Ecrit le roi dans le fichier
        bbox_value = hands[0].get('bbox')
        new_image = img_copy[bbox_value[1]:bbox_value[1] + bbox_value[3], bbox_value[0]:bbox_value[0] + bbox_value[2]]

        #Prétraitements identique aux images d'entraînement
        new_image = cv2.GaussianBlur(new_image, (5, 5), 0)
        new_image = cv2.resize(new_image, (50, 50))
        new_image = np.array(new_image) / 255.0
        new_image = np.expand_dims(new_image, axis=0)

        #predictions
        predictions = loaded_model.predict(new_image)
        class_index = np.argmax(predictions[0])

        # print(predictions)
        results = predictions[0].tolist()
        print(results)
        print("Predicted class index:", class_index,classData[class_index])
        #Affichage du texte
        cv2.putText(img, f"signe : {classData[class_index]} [{(results[class_index]):.2f}%]", (30, 30), cv2.FONT_HERSHEY_PLAIN,2, (255, 0, 0), 2)
        
    #Affichage de l'image
    cv2.imshow("Image", img)

capture.release()
cv2.destroyAllWindows()

    